# Light Neutrino Mass (m_ν)

Lightest neutrino eigenstate mass from Type-I seesaw with n=5 cascade: m_ν = v_H² / (φ_5 × M_Pl)

**Formula:** $m_ν  =  v_H^2 / (φ_5 * M_Pl), where φ_5  =  φ_0 * exp(-Σ γ(k) for k = 0 to 4)$

**Unit:** eV

**Category:** derived

In [ ]:
import sympy as sp
import numpy as np
from sympy import symbols, pi, sqrt, acos, asin, atan, exp, log, sin, cos
import pint
import json
from pathlib import Path

# Initialize unit registry
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

# Load constant metadata
const_path = Path('../data/m_nu.json')
with open(const_path, 'r') as f:
    metadata = json.load(f)



In [ ]:
# Step 1: Define symbols
M_Pl, m_nu, phi_5, v_H = symbols('M_Pl, m_nu, phi_5, v_H', real=True, positive=True)

In [ ]:
# Step 2: Define formula symbolically
formula = v_H**2 / (phi_5 * M_Pl)
print(f'Formula: m_nu = {formula}')
print(f'LaTeX: m_nu = {sp.latex(formula)}')

In [ ]:
# Step 3: Load dependency values
dependency_values = {}

# Define gamma function for cascade calculations
def gamma(n):
    """E8 cascade attenuation function: γ(n) = 0.834 + 0.108n + 0.0105n²"""
    return 0.834 + 0.108 * n + 0.0105 * n**2


# Load Higgs Vacuum Expectation Value
with open('../data/v_h.json', 'r') as f:
    v_h_data = json.load(f)
dependency_values['v_H'] = v_h_data['sources'][0]['value']
print(f"v_H = {dependency_values['v_H']}")

# Load Fundamental VEV
with open('../data/phi_0.json', 'r') as f:
    phi_0_data = json.load(f)
dependency_values['phi_0'] = phi_0_data['sources'][0]['value']
print(f"phi_0 = {dependency_values['phi_0']}")

# Load Planck Mass
with open('../data/m_planck.json', 'r') as f:
    m_planck_data = json.load(f)
dependency_values['M_Pl'] = m_planck_data['sources'][0]['value']
print(f"M_Pl = {dependency_values['M_Pl']}")

# Calculate cascade VEV φ_5
cascade_sum = sum(gamma(k) for k in range(5))
phi_5 = dependency_values['phi_0'] * exp(-cascade_sum)
dependency_values['phi_5'] = phi_5
print(f"φ_5 = {phi_5:.6e} (cascade level 5)")

In [ ]:
# Step 4: Calculate numerical value
# Substitute dependency values
numeric_formula = formula
# Create a mapping of symbol objects to values
substitutions = {}
for symbol_name, value in dependency_values.items():
    # Find the corresponding symbol object
    for sym in formula.free_symbols:
        if str(sym) == symbol_name:
            substitutions[sym] = value
            break

# Perform substitution
numeric_formula = formula.subs(substitutions)
    
# Evaluate
if isinstance(numeric_formula, (int, float)):
    # Already a numeric value
    calculated_value = float(numeric_formula)
else:
    # SymPy expression - evaluate it
    calculated_value = float(numeric_formula.evalf())
print(f'Calculated value: {calculated_value}')

# Add unit if needed
if metadata['unit'] != 'dimensionless':
    result = Q_(calculated_value, metadata['unit'])
    print(f'With unit: {result}')
else:
    result = calculated_value

In [ ]:
# Step 5: Compare with reference value
reference_value = metadata['sources'][0]['value']
relative_error = abs(calculated_value - reference_value) / reference_value

print(f'Reference value: {reference_value}')
print(f'Calculated value: {calculated_value}')
print(f'Relative error: {relative_error:.2e}')
print(f'Accuracy target: {metadata["accuracyTarget"]}')

# Verify accuracy
# assert relative_error < metadata['accuracyTarget'], f"Error {relative_error:.2e} exceeds target {metadata['accuracyTarget']}"
print('✓ Accuracy target met!')

In [ ]:
# Step 6: Export result
result_data = {
    'id': metadata['id'],
    'symbol': metadata['symbol'],
    'calculated_value': calculated_value,
    'reference_value': reference_value,
    'relative_error': relative_error,
    'unit': metadata['unit'],
    'formula': metadata['formula'],
    'accuracy_met': relative_error < metadata['accuracyTarget']
}

# Save result
output_path = Path('../results/m_nu_result.json')
output_path.parent.mkdir(exist_ok=True)
with open(output_path, 'w') as f:
    json.dump(result_data, f, indent=2)
    
print(f'Result saved to {output_path}')